In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.optim import lr_scheduler
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

import math, os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from random import randint
from sklearn.datasets import fetch_mldata


In [2]:
mnist = fetch_mldata("MNIST original")

TRAINDATA = mnist['data'][:60000]/255.0
TESTDATA = mnist['data'][60000:]/255.0
TRAINLABELS = mnist['target'][:60000]
TESTLABELS = mnist['target'][60000:]

# get validation set from training
listIdxs = range(TRAINLABELS.shape[0])
np.random.shuffle(listIdxs)
np.random.shuffle(listIdxs)
VALDATA = TRAINDATA[listIdxs[:6000]]
VALLABELS = TRAINLABELS[listIdxs[:6000]]

## Network Definitions

In [3]:
'''
Overarching Model
'''

class CapsNet(nn.Module):
    def __init__(self, routing_iterations, 
                 preCapDepth, numCaps, numPerCap, capFilterSize,
                c1kernelSizes, c1kernelStrides, c1numMaps, n_classes=10):
        super(CapsNet, self).__init__()
        self.preCapDepth = preCapDepth
        self.numCaps = numCaps
        self.numPerCap = numPerCap
        self.c1kernelSizes = c1kernelSizes
        self.c1kernelStrides = c1kernelStrides
        self.c1numMaps = c1numMaps
        
        #self.conv1 = nn.Conv2d(1, 256, kernel_size=9, stride=1)
        self.conv1 = PrimaryConvs(self.c1kernelSizes, self.c1kernelStrides,self.c1numMaps)
        
        #self.primaryCaps = PrimaryCapsLayer(self.preCapDepth, self.m=numCaps, self.numPerCap, kernel_size=9, stride=2)  # outputs: normalized sheets
        self.primaryCaps = PrimaryCapsLayer(self.preCapDepth, self.numCaps, self.numPerCap)
        
        self.num_primaryCaps = numCaps * (capFilterSize**2)
        routing_module = AgreementRouting(self.num_primaryCaps, n_classes, routing_iterations)
        self.digitCaps = CapsLayer(self.num_primaryCaps, self.numPerCap, n_classes, 16, routing_module)

    def forward(self, input):
        x = self.conv1(input)
        x = self.primaryCaps(x) # output sheet
        x = self.digitCaps(x) # output score matrix (before final flat)
        probs = x.pow(2).sum(dim=2).sqrt()
        return x, probs

### Input Networks

In [4]:
'''
Input: FM after 1st convolution
Output: Normalized sheet
'''
class PrimaryCapsLayer(nn.Module):
    def __init__(self, input_channels, output_caps, output_dim):
        super(PrimaryCapsLayer, self).__init__()
        #self.conv = nn.Conv2d(input_channels, output_caps * output_dim, kernel_size=kernel_size, stride=stride)
        self.input_channels = input_channels
        self.output_caps = output_caps
        self.output_dim = output_dim

    def forward(self, input):
        #out = self.conv(input)
        out = input
        N, C, H, W = out.size()
        out = out.view(N, self.output_caps, self.output_dim, H, W) # splitting into capsules

        # will output N x OUT_CAPS x OUT_DIM
        out = out.permute(0, 1, 3, 4, 2).contiguous() 
        out = out.view(out.size(0), -1, out.size(4)) # flat N*sheet
        out = squash(out) # normalize
        return out
    
'''
Input: FM after 1st convolution
Output: Normalized sheet
'''
class PrimaryConvs(nn.Module):
    def __init__(self, c1kernelSizes, c1kernelStrides, c1numMaps):
        super(PrimaryConvs, self).__init__()
        self.c1kernelSizes = c1kernelSizes
        self.c1kernelStrides = c1kernelStrides
        self.c1numMaps = c1numMaps
        self.numLayers = len(c1numMaps)
        
        self.conv1 = None
        if self.numLayers >= 1:
            self.conv1 = nn.Conv2d(1, self.c1numMaps[0], 
                                   kernel_size=self.c1kernelSizes[0], 
                                   stride=self.c1kernelStrides[0])
        self.conv2 = None
        if self.numLayers >= 2:
            self.conv2 = nn.Conv2d(self.c1numMaps[0], self.c1numMaps[1], 
                                   kernel_size=self.c1kernelSizes[1], 
                                   stride=self.c1kernelStrides[1])
        self.conv3 = None
        if self.numLayers >= 3:
            self.conv3 = nn.Conv2d(self.c1numMaps[1], self.c1numMaps[2], 
                                   kernel_size=self.c1kernelSizes[2], 
                                   stride=self.c1kernelStrides[2])
        self.conv4 = None
        if self.numLayers >= 4:
            self.conv4 = nn.Conv2d(self.c1numMaps[2], self.c1numMaps[3], 
                                   kernel_size=self.c1kernelSizes[3], 
                                   stride=self.c1kernelStrides[3])
    def forward(self, x):
        if self.conv1:
            x = self.conv1(x)
            if self.numLayers != 1:
                x = F.relu(x)
        if self.conv2:
            x = self.conv2(x)
            if self.numLayers != 2:
                x = F.relu(x)
        if self.conv3:
            x = self.conv3(x)
            if self.numLayers != 3:
                x = F.relu(x)
        if self.conv4:
            x = self.conv4(x)
            if self.numLayers != 4:
                x = F.relu(x)
        return x
        

In [5]:
def squash(x):
    lengths2 = x.pow(2).sum(dim=2)
    lengths = lengths2.sqrt()
    x = x * (lengths2 / (1 + lengths2) / lengths).view(x.size(0), x.size(1), 1)
    return x


class AgreementRouting(nn.Module):
    def __init__(self, input_caps, output_caps, n_iterations):
        super(AgreementRouting, self).__init__()
        self.n_iterations = n_iterations
        self.b = nn.Parameter(torch.zeros((input_caps, output_caps)))

    def forward(self, u_predict):
        batch_size, input_caps, output_caps, output_dim = u_predict.size()
        
        c = F.softmax(self.b)
        s = (c.unsqueeze(2) * u_predict).sum(dim=1)
        v = squash(s)

        if self.n_iterations > 0:
            b_batch = self.b.expand((batch_size, input_caps, output_caps))
            for r in range(self.n_iterations):
                v = v.unsqueeze(1)
                b_batch = b_batch + (u_predict * v).sum(-1)

                c = F.softmax(b_batch.view(-1, output_caps)).view(-1, input_caps, output_caps, 1)
                s = (c * u_predict).sum(dim=1)
                v = squash(s)
        return v


class CapsLayer(nn.Module):
    def __init__(self, input_caps, input_dim, output_caps, output_dim, routing_module):
        super(CapsLayer, self).__init__()
        self.input_dim = input_dim
        self.input_caps = input_caps
        self.output_dim = output_dim
        self.output_caps = output_caps
        self.weights = nn.Parameter(torch.Tensor(input_caps, input_dim, output_caps * output_dim))
        self.routing_module = routing_module
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.input_caps)
        self.weights.data.uniform_(-stdv, stdv)

    def forward(self, caps_output):
        caps_output = caps_output.unsqueeze(2)
        u_predict = caps_output.matmul(self.weights)
        u_predict = u_predict.view(u_predict.size(0), self.input_caps, self.output_caps, self.output_dim)
        v = self.routing_module(u_predict)
        return v


In [6]:
### Reconstruction Stuff 
class ReconstructionNet(nn.Module):
    def __init__(self, n_dim=16, n_classes=10):
        super(ReconstructionNet, self).__init__()
        self.fc1 = nn.Linear(n_dim * n_classes, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, 784)
        self.n_dim = n_dim
        self.n_classes = n_classes

    def forward(self, x, target):
        mask = Variable(torch.zeros((x.size()[0], self.n_classes)), requires_grad=False)
        if next(self.parameters()).is_cuda:
            mask = mask.cuda()
        mask.scatter_(1, target.view(-1, 1), 1.)
        mask = mask.unsqueeze(2)
        x = x * mask
        x = x.view(-1, self.n_dim * self.n_classes)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x


class CapsNetWithReconstruction(nn.Module):
    def __init__(self, capsnet, reconstruction_net):
        super(CapsNetWithReconstruction, self).__init__()
        self.capsnet = capsnet
        self.reconstruction_net = reconstruction_net

    def forward(self, x, target):
        x, probs = self.capsnet(x)
        reconstruction = self.reconstruction_net(x, target)
        return reconstruction, probs

### Data Sets

In [7]:
class GeneralDataset(Dataset):
    def __init__(self,data,labels):
        self.data = [torch.DoubleTensor(d).view(28,28) for d in data]
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

TRAINDATASET = GeneralDataset(TRAINDATA,TRAINLABELS)
VALDATASET = GeneralDataset(VALDATA,VALLABELS)
TESTDATASET = GeneralDataset(TESTDATA,TESTLABELS)


## Train

### Helper functions

In [8]:
### Define Loss Functions
class MarginLoss(nn.Module):
    def __init__(self, m_pos, m_neg, lambda_):
        super(MarginLoss, self).__init__()
        self.m_pos = m_pos
        self.m_neg = m_neg
        self.lambda_ = lambda_

    def forward(self, lengths, targets, size_average=True):
        t = torch.zeros(lengths.size()).long()
        if targets.is_cuda:
            t = t.cuda()
        t = t.scatter_(1, targets.data.view(-1, 1), 1)
        targets = Variable(t)
        losses = targets.float() * F.relu(self.m_pos - lengths).pow(2) + \
                 self.lambda_ * (1. - targets.float()) * F.relu(lengths - self.m_neg).pow(2)
        return losses.mean() if size_average else losses.sum()

### save model
def saveModel(model,name,state_dict=False):
    if state_dict:
        torch.save(model.state_dict(),name)
    else:
        torch.save(model,name)


### rotation transform class
class RandomRotation(object):
    def __init__(self, degrees_tup):
        self.dt = [0]*6
        self.dt[degrees_tup] = 1

    def __call__(self, sample):
        
        if self.dt[0]:
            return sample
        sample = np.asarray(sample)
       
        if self.dt[0]:
            sample = rotate(randint(0, 1),sample)
            return np2PIL(sample)
        if self.dt[1]:
            sample = rotate(randint(0, 1)*2,sample)
            return np2PIL(sample)
        if self.dt[2]:
            sample = rotate(randint(0, 1)*3,sample)
            return np2PIL(sample)
        if self.dt[3]:
            sample = rotate(randint(1, 3),sample)
            return np2PIL(sample)
        if self.dt[4]:
            sample = rotate(randint(0, 3),sample)
            return np2PIL(sample)
def np2PIL(data):
    return Image.fromarray(data)
def rotate(i,data):
    if i == 0:
        return data
    elif i == 1:
        return np.rot90(data)
    elif i == 2:
        return np.rot90(data,2)
    elif i == 3:
        return np.rot90(data,3)
    
### Normalize
def normalize(tensorData):
    return torch.Tensor(tensorData.numpy()/255.)

In [9]:
def train(Epoch,Loss,numStatsPerEpoch,Rec_loss_every=1):
    model.train()
    statIdx = [len(TRAINLOADER)/numStatsPerEpoch]*numStatsPerEpoch
    statIdx = [(i)*a for i,a in enumerate(statIdx)]
    for batch_idx, (data, target) in enumerate(TRAINLOADER):
        
        # get stats
        if (batch_idx) in statIdx:
            val_loss, val_acc,VN,VD = getLossAccND('Validation',VALLOADER)
            test_loss, test_acc,TN,TD = getLossAccND('Test',TESTLOADER)
            print
            ValLoss.append(val_loss);ValAcc.append(val_acc);
            TestLoss.append(test_loss);TestAcc.append(test_acc);
    
        # train
        data = data.view(-1,1,28,28)
        data, target = data.float(), target.long()
        if CUDA:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target, requires_grad=False)
        optimizer.zero_grad()
        if RECONSTRUCTION:
            output, probs = model(data, target)
            reconstruction_loss = F.mse_loss(output, data.view(-1, 784))
            margin_loss = loss_fn(probs, target)
            loss = reconstruction_alpha * reconstruction_loss + margin_loss
        else:
            output, probs = model(data)
            loss = loss_fn(probs, target)
        
        if batch_idx % Rec_loss_every == 0:
            Loss.append(loss.data[0])
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INT == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                Epoch, batch_idx * len(data), len(TRAINLOADER.dataset),
                       100. * batch_idx / len(TRAINLOADER), loss.data[0]))

def getLossAccND(name,dataloader):
    model.eval()
    loss = 0
    correct = 0
    for data, target in dataloader:
        data = data.view(-1,1,28,28)
        data, target = data.float(), target.long()
        if CUDA:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        #print data, target
        if RECONSTRUCTION:
            output, probs = model(data, target)
            reconstruction_loss = F.mse_loss(output, data.view(-1, 784), size_average=False).data[0]
            loss += loss_fn(probs, target, size_average=False).data[0]
            loss += reconstruction_alpha * reconstruction_loss
        else:
            output, probs = model(data)
            loss += loss_fn(probs, target, size_average=False).data[0]

        pred = probs.data.max(1, keepdim=True)[1]  # get the index of the max probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    loss /= len(dataloader.dataset)
    print('{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        name,loss, correct, len(dataloader.dataset),
        100. * correct / len(dataloader.dataset)))
    return (loss, 100. * correct / len(dataloader.dataset), correct, len(dataloader.dataset))

### Defining paramters, loaders, SGD elements

In [17]:
# # Training settings
# parser = argparse.ArgumentParser(description='CapsNet with MNIST')
# parser.add_argument('--batch-size', type=int, default=128, metavar='N',
#                     help='input batch size for training (default: 64)')
# parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#                     help='input batch size for testing (default: 1000)')
# parser.add_argument('--epochs', type=int, default=250, metavar='N',
#                     help='number of epochs to train (default: 10)')
# parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
#                     help='learning rate (default: 0.01)')
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='disables CUDA training')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                     help='random seed (default: 1)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='how many batches to wait before logging training status')
# parser.add_argument('--routing_iterations', type=int, default=3)
# parser.add_argument('--with_reconstruction', action='store_true', default=False)
# args = parser.parse_args()
# args.cuda = not args.no_cuda and torch.cuda.is_available()

### Model Architecture Params
ROUTING_ITERS = 3
CONVKERNELSIZES, CONVKERNELSTRIDES, CONVNUMMAPS = \
    [9],[2],[256]
PRECAPDEPTH = CONVNUMMAPS[-1]
NUMCAPS = 32
NUMPERCAP = 8  #Note: precapdepth=numcaps*numpercap
CAPFILTERSIZE = 10


### Training Params
BATCH_SIZE = 200
TEST_BATCH_SIZE = 400
EPOCHS = 10
LEARNING_RATE = 0.001
DECAY_EPOCH = 3
CUDA = torch.cuda.is_available()
SEED = 1
LOG_INT = 10 # how many batches to wait before logging training status
RECONSTRUCTION = False
ROTATION_TYPE = 3
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}

### Stats
NUMSTATSPEREPOCH = 4


TRAINLOADER = DataLoader(TRAINDATASET, batch_size=BATCH_SIZE,
                            shuffle=True, num_workers=4)
VALLOADER = DataLoader(VALDATASET, batch_size=TEST_BATCH_SIZE,
                            shuffle=False, num_workers=4) 
TESTLOADER = DataLoader(TESTDATASET, batch_size=TEST_BATCH_SIZE,
                            shuffle=False, num_workers=4) 

model = CapsNet(ROUTING_ITERS,
                PRECAPDEPTH, NUMCAPS, NUMPERCAP, CAPFILTERSIZE,
                CONVKERNELSIZES,CONVKERNELSTRIDES,CONVNUMMAPS,
                n_classes=10)
#model = CapsNet( ROUTING_ITERS,PRECAPDEPTH, NUMCAPS, NUMPERCAP, CAPFILTERSIZE,
#               CONVKERNELSIZES, CONVKERNELSTRIDES, CONVNUMMAPS )

if RECONSTRUCTION:
    reconstruction_model = ReconstructionNet(16, 10)
    reconstruction_alpha = 0.0005
    model = CapsNetWithReconstruction(model, reconstruction_model)

if CUDA:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
loss_fn = MarginLoss(0.9, 0.1, 0.5)

In [18]:
### Call training functions 
TrainLoss = []; TrainL_every = 1; TrN=0; TrD=0
ValLoss = []; ValAcc = []; VN=0;VD=0
TestLoss = []; TestAcc = []; TN=0; TD=0
for epoch in range(1, EPOCHS + 1):
    train(epoch,TrainLoss,NUMSTATSPEREPOCH,TrainL_every)
    scheduler.step(TestLoss[-1])
# get final training and testing stats
ftest_loss, ftest_acc,TN,TD = getLossAccND('Test',TESTLOADER)
val_loss, val_acc,VN,VD = getLossAccND('Validation',VALLOADER)
ftrain_loss, ftrain_acc,TrN,TrD = getLossAccND('Train',TRAINLOADER)

Validation set: Average loss: 0.7984, Accuracy: 723/6000 (12%)
Test set: Average loss: 0.7982, Accuracy: 1230/10000 (12%)



RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1503966894950/work/torch/lib/THC/generic/THCStorage.cu:66

In [ ]:
# initialize network classes
#capsnet = CapsNet(3, 10)
#reconstructionnet = ReconstructionNet(16, 10)
#model = CapsNetWithReconstruction(capsnet, reconstructionnet)

In [ ]:
plt.title('Training Loss')
plt.plot(TrainLoss)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()
print 'Final Training Accuracy: %d/%d (%.2f%%)' % (TrN,TrD,ftrain_acc)
print 'Final Training Loss: %.5f' % (ftrain_loss)

statsIdx = range(len(ValAcc))
statsIdx = [float(c)/NUMSTATSPEREPOCH for c in statsIdx]

fig, ax1 = plt.subplots()
ax1.plot(statsIdx,ValLoss, 'b-')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='b')
ax1.tick_params('y', colors='b')
ax2 = ax1.twinx()
ax2.plot(statsIdx,ValAcc, 'r-')
ax2.set_ylabel('Accuracy', color='r')
ax2.tick_params('y', colors='r')
fig.tight_layout()
plt.title('Validation Loss and Accuracy')
plt.show()
print 'Final Validation Accuracy: %d/%d (%.2f%%)' % (VN,VD,ValAcc[-1])
print 'Final Validation Loss: %.5f' % (ValLoss[-1])

fig, ax1 = plt.subplots()
ax1.plot(statsIdx,TestLoss, 'b-')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='b')
ax1.tick_params('y', colors='b')
ax2 = ax1.twinx()
ax2.plot(statsIdx,TestAcc, 'r-')
ax2.set_ylabel('Accuracy', color='r')
ax2.tick_params('y', colors='r')
fig.tight_layout()
plt.title('Test Loss and Accuracy')
plt.show()
print 'Final Test Accuracy: %d/%d (%.2f%%)' % (TN,TD,TestAcc[-1])
print 'Final Test Loss: %.5f' % (TestLoss[-1])

